## CMS document

### HIN 2021 tracking selection development
The version of cmssw can be checked by
```bash
scram list
```
Setupt the workspace
```bash
scram p -n hi2021_runPrep_11_1_0_pre6 CMSSW_11_1_0_pre6
cd hi2021_runPrep_11_1_0_pre6/src
cmsenv
scram b -j4
```

### Generating the sample

We first need to generate the minbias samples as background and embed the QCD jet sample into it.

For generating the minbias sample, the command is
```bash
cmsDriver.py Hydjet_Quenched_B12_5020GeV_cfi  --conditions auto:phase1_2021_realistic_hi -n 1 --era Run3_pp_on_PbPb --eventcontent RAWSIM --relval 2000,1 -s GEN,SIM --datatier GEN-SIM --beamspot RealisticPbPbCollision2018 --geometry DB:Extended
```
where the beamspot used here is "RealisticPbPbCollision2018" and need to turn the `cFlag` on after generated the cfg file manually (otherwise, it will only generate the sample with b in the range specified in cfg).

To generate the QCD jets with Pthat 80-120 (`QCD_Pt_80_120_14TeV_TuneCP5_cfi`) embedding samples (using the selfgenerated sample for embedding):
```bash
cmsDriver.py QCD_Pt_80_120_14TeV_TuneCP5_cfi --mc --pileup HiMixGEN --pileup_input "dbs:/MinBias/wangx-minBias_HydjetDrum5F_GEN_SIM_PU-43543e2e0073ceac48fd2078951ca5eb/USER instance=prod/phys03" --eventcontent RAWSIM --datatier GEN-SIM --conditions auto:phase1_2021_realistic_hi --beamspot RealisticPbPbCollision2018 --step GEN,SIM --scenario HeavyIons --geometry DB:Extended --era Run3_pp_on_PbPb -n 2
```

The RECO configure can be generated as
```bash
cmsDriver.py step2  --conditions auto:phase1_2021_realistic_hi -s DIGI:pdigi_hi,L1,DIGI2RAW,HLT:@fake2 --datatier GEN-SIM-DIGI-RAW-HLTDEBUG --pileup HiMixNoPU -n 2 --era Run3_pp_on_PbPb --eventcontent FEVTDEBUGHLT

cmsDriver.py step3  --runUnscheduled  --conditions auto:phase1_2021_realistic_hi -s RAW2DIGI,L1Reco,RECO --datatier GEN-SIM-RECODEBUG -n 2 --era Run3_pp_on_PbPb --eventcontent RECODEBUG
```


|type|stats|data tier| PD name|
|---|---|---|---|
|minbias|10k|GEN | `/MinBias/wangx-minBias_HydjetDrum5F_GEN_SIM_PU-43543e2e0073ceac48fd2078951ca5eb/USER instance=prod/phys03`|
|QCD jet [80,120]|10k|GEN|`/EmbeddedSample/wangx-QCD_Pt_80_120_embedding_HydjetDrum5F_GEN_SIM_PU-cc5c0c339fd6ec7446e66c9541a0548e/USER instance=prod/phys03`|

## Tracking selection
Setup the tracking code in the subdirectory under the `$CMSSW_BASE/src/`
```bash
git clone git@github.com:Taburis/MVATracking.git
```
Preparing the reconstruction cfg by
```bash
cmsDriver.py step3  --runUnscheduled  --conditions auto:phase1_2021_realistic_hi -s RAW2DIGI,L1Reco,RECO --datatier GEN-SIM-RECO -n 2 --era Run3_pp_on_PbPb --eventcontent FEVTDEBUGHLT
```

The `TMVA` are not supported by the GForest anymore, some modification are needed.